In [1]:
import csv
import pandas as pd
import random
import re
import time
from tqdm import tqdm, trange

In [2]:
df = pd.read_csv('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/inputTable.csv')
df.head()

,paperA,paperB,A_firstAuthorInDegree,A_firstAuthorOutDegree,A_PaperOutDegree,abstractSim,notHasCitationPurpose,ifFirstAuthorInSameAffilaition,ifSameJournal,selfCite,citedPaperJournalTypeProportionInCitingPaperReferences,citedPaperFirstAuthorHasCiteCitingPaperFirstAuthorsPaper,citingPaperAuthorsPapersIntersectionCitedPaperAuthorsPapers,B_firstAuthorInDegree,B_firstAuthorOutDegree,B_PaperOutDegree,label_ifAbnormal
0,2138715703,2007600326,0,171,28,0.637714,0.672244,0,0,0,0.027027,0,0,6,0,0,0
1,2138715703,2147617783,0,171,28,0.569197,0.723836,0,0,0,0.054054,0,0,10,0,0,0
2,2138715703,1966687480,0,171,28,0.473437,0.770526,0,0,0,0.027027,0,0,6,0,0,0
3,2138715703,2062460839,0,171,28,0.572876,0.651287,0,0,0,0.027027,0,0,6,0,0,0
4,2138715703,2134441116,0,171,28,0.524535,0.755068,1,0,0,0.054054,1,1,22,0,0,0


In [3]:
df_t = df.loc[df['label_ifAbnormal'] == 0]
df_f = df.loc[df['label_ifAbnormal'] != 0]

In [4]:
paperaidlist = []
paperaidlist = df_t['paperA'].unique()
paperaidliststr = [str(x) for x in paperaidlist]

In [5]:
paperbidlist = []
paperbidlist = df_t['paperB'].unique()
paperbidliststr = [str(x) for x in paperbidlist]

In [6]:
f2 = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/allPaperAbstracts-doc2vec-Embedding-128.txt','r')
paperid_abstractdoc2vec = []
for line in f2:
    line = line.strip().split('\t')
    paperid_abstractdoc2vec.append(line[0])

# core paper 和 true cited paper

In [13]:
random.seed(3)
randompaperidliststr = random.sample(paperaidliststr,50)
#have vectors
corepaperidliststr_invec = [x for x in randompaperidliststr if x in paperid_abstractdoc2vec]
print(len(corepaperidliststr_invec))

47


In [14]:
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/Papers.txt','r')
alljournalidOf_corepaperidliststr_invec = []
allpaperidOf_corepaperidliststr_invec = []
for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[0] in corepaperidliststr_invec:
        allpaperidOf_corepaperidliststr_invec.append(line[0])
        alljournalidOf_corepaperidliststr_invec.append(line[10])

172037947it [09:02, 316834.20it/s]


In [10]:
allvalidjournalid = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/selectedJournals.txt','r')
for line in tqdm(f) :
    line = line.strip().split('\t')
    allvalidjournalid.append(line[0])
print(len(allvalidjournalid))

7623it [00:00, 1108947.68it/s]

7623


In [16]:
#删掉没有journalid的论文
print(len(alljournalidOf_corepaperidliststr_invec))#47
corepaperidliststr = []
for paperid,journalid in zip(allpaperidOf_corepaperidliststr_invec,alljournalidOf_corepaperidliststr_invec):
    if journalid in allvalidjournalid:
        corepaperidliststr.append(paperid)
print(len(corepaperidliststr)) #38
corepaperidlist = [int(x) for x in corepaperidliststr]
print(len(corepaperidlist)) ##38

47
38
38


In [17]:
#真实被引用的paper，去掉没有 abstract vec的
paperidcitedbycorepaper = []
df_citedpaper = df[df['paperA'].isin(corepaperidlist)]
print(len(df_citedpaper)) #178
paperidcitedbycorepaperori = df_citedpaper['paperB'].unique()
print(len(paperidcitedbycorepaperori)) #166
paperidcitedbycorepaperoristr = [str(x) for x in paperidcitedbycorepaperori]
paperbinabstractem = [x for x in paperidcitedbycorepaperoristr if x in paperid_abstractdoc2vec]
print(len(paperbinabstractem)) #161
paperbinabstractem_uncore = [x for x in paperbinabstractem if x not in corepaperidliststr]
print(len(paperbinabstractem_uncore))#160

178
166
161
160


In [20]:

f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/Papers.txt','r')
alljournalidOf_citedpaperidliststr_invec = []
allpaperidOf_citedpaperidliststr_invec = []
for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[0] in paperbinabstractem_uncore:
        allpaperidOf_citedpaperidliststr_invec.append(line[0])
        alljournalidOf_citedpaperidliststr_invec.append(line[10])
f.close()

172037947it [14:14, 201402.03it/s]


In [21]:
#删掉没有journalid的论文
print(len(alljournalidOf_citedpaperidliststr_invec))#160
citedpaperidliststr = []
for paperid,journalid in zip(allpaperidOf_citedpaperidliststr_invec,alljournalidOf_citedpaperidliststr_invec):
    if journalid in allvalidjournalid:
        citedpaperidliststr.append(paperid)
print(len(citedpaperidliststr)) #137
paperidcitedbycorepaperstr = citedpaperidliststr
print(len(paperidcitedbycorepaperstr))#137
paperidcitedbycorepaper = [int(x) for x in paperidcitedbycorepaperstr]
print(len(paperidcitedbycorepaper)) ##137

160
137
137
137


In [22]:
#删掉core paper中重复的
paperidcitedbycorepaper = sorted(set(paperidcitedbycorepaper))
print(len(paperidcitedbycorepaper)) #137
paperidcitedbycorepaper = [x for x in paperidcitedbycorepaper if x not in corepaperidlist]
print(len(paperidcitedbycorepaper)) #137
paperidcitedbycorepaperstr = [str(x) for x in paperidcitedbycorepaper]
print(len(paperidcitedbycorepaperstr))# 137

137
137
137


# 需要添加的异常引用的paper id+所需要用到的所有paper id

In [23]:
#把core paper分两半，后一半加入异常引用
print(len(corepaperidliststr)) #38
corepaperidliststr1 = corepaperidliststr[0:19]
print(len(corepaperidliststr1))  #19
corepaperidliststr2 = corepaperidliststr[19:38]
print(len(corepaperidliststr2))  #19
corepaperidlist1 = corepaperidlist[0:19]
print(len(corepaperidlist1))  #19
corepaperidlist2 = corepaperidlist[19:38]
print(len(corepaperidlist2))  #19

38
19
19
19
19


In [24]:
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/Papers.txt','r')
alljournalidOf_2nd = []
for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[0] in corepaperidliststr2:
        alljournalidOf_2nd.append(line[10])
f.close()

172037947it [08:05, 354697.54it/s]


In [25]:
#delete unvalid journal
annmous_journalstr = [x for x in alljournalidOf_2nd if x in allvalidjournalid]

In [27]:
#papers published by annmous journal
annmous_paperid_ori = []
annmous_journalid_ori = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/Papers.txt','r')

for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[10] in annmous_journalstr:
        annmous_paperid_ori.append(line[0])
        annmous_journalid_ori.append(line[10])
f.close()

172037947it [08:02, 356522.28it/s]


In [28]:
print(len(annmous_paperid_ori))#356974
annmous_paperid0 = sorted(set(annmous_paperid_ori))
print(len(annmous_paperid0))#356974
annmous_paperid1 = [x for x in annmous_paperid0 if x in paperid_abstractdoc2vec]
print(len(annmous_paperid1))#249
annmous_paperid2 = [x for x in annmous_paperid1 if x not in corepaperidliststr]
print(len(annmous_paperid2))#228
annmous_paperid = [x for x in annmous_paperid2 if x not in paperidcitedbycorepaperstr]
print(len(annmous_paperid))#215

356974
356974
249
228
215


In [29]:
#所有论文id
allpaperidstr = corepaperidliststr + paperidcitedbycorepaperstr + annmous_paperid
print(len(allpaperidstr)) #390 + 38 + 137 + 215
allpaperid = [int(x) for x in allpaperidstr]
print(len(allpaperid))

390
390


In [30]:

with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed/allpaperid.txt', 'w') as cf:
    for line in allpaperidstr:
        cf.write(line+'\n')

cf.close()

# journal和paper-journal

In [31]:
journal = alljournalidOf_corepaperidliststr_invec + alljournalidOf_citedpaperidliststr_invec + annmous_journalid_ori
paper = allpaperidOf_corepaperidliststr_invec + allpaperidOf_citedpaperidliststr_invec + annmous_paperid_ori

alljournalid = []
publishedpaper = []
for item1,item2 in zip(journal,paper):
    if item2 in allpaperidstr:
        alljournalid.append(item1)
        publishedpaper.append(item2)
        
print(len(alljournalid)) #424
#alljournalid = sorted(set(alljournalid))
print(len(publishedpaper)) #424


424
424


In [32]:
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed/paperjournal.txt', 'w') as cf:
    for p,j in zip(publishedpaper,alljournalid):
        cf.write(p + ' ' + j + '\n')

cf.close()

In [38]:
alljournal = sorted(set(alljournalid))
print(len(alljournal))
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed//alljournalid.txt', 'w') as cf:
    for line in alljournal:
        cf.write(line+'\n')

cf.close()

102


# author 和author-paper

In [39]:
paperid_inpaperauthor = []
authorid_inpaperauthor = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/PaperAuthorAffiliations.txt','r')

for line in tqdm(f):
    line = line.split('\t')
    if line[0] in allpaperidstr:
        paperid_inpaperauthor.append(line[0]) #str
        authorid_inpaperauthor.append(line[1]) #str    
print(len(paperid_inpaperauthor)) #588
print(len(authorid_inpaperauthor)) #753
f.close()

455999370it [45:07, 168405.42it/s]

2759
2759


In [40]:
allauthor = sorted(set(authorid_inpaperauthor))
print(len(allauthor)) #2566
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed/allauthorid.txt', 'w') as cf:
    for line in allauthor:
        cf.write(line+'\n')

cf.close()

2566


In [41]:
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed/paperauthor.txt', 'w') as cf:
    for p,a in zip(paperid_inpaperauthor,authorid_inpaperauthor):
        cf.write(p + ' ' + a + '\n')

cf.close()

# citing relationship

In [43]:
citinginfo = df_t.loc[df_t['paperA'].isin(corepaperidlist)]
paperidcitedbycorepaper = [int(x) for x in paperidcitedbycorepaperstr]
citinginfo1 = citinginfo.loc[citinginfo['paperB'].isin(paperidcitedbycorepaper)]
citingpaperid = list(citinginfo1['paperA'])
citedpaperid = list(citinginfo1['paperB'])

for id1 in corepaperidliststr2:
    journal = []
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed/paperjournal.txt', 'r')
    for line in f:
        line = line.strip().split(' ')
        if line[0] == id1:
            journal.append(line[1])
    #print(len(author))
    paper = []
    for jj in journal:
        
        f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed/paperjournal.txt', 'r')
        for line in f:
            line = line.strip().split(' ')
            if line[1] == jj:
                paper.append(line[0])
        #print(len(paper))
    paper = sorted(set(paper))
    paper = [x for x in paper if x != id1]
    for i in range(len(paper)):
        citingpaperid.append(int(id1))
        citedpaperid.append(int(paper[i]))
f.close()

In [44]:
print(len(citinginfo1))#真实引用数量 143
print(len(citingpaperid))#总引用数量 545
a = len(citinginfo1)
b = len(citingpaperid)
c = b - a
print(c) #402

143
545
402


In [45]:
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed/citinginfo.txt', 'w') as cf:
    for ing,ed in zip(citingpaperid, citedpaperid):
        cf.write(str(ing) + ' ' + str(ed) + '\n')

cf.close()

# abstract vec matrix

In [46]:
embeddingvectors = []
x = 0
print(len(allpaperidstr))#390
for ids in allpaperidstr:
    f2 = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/allPaperAbstracts-doc2vec-Embedding-128.txt','r')
    for line in f2:
        line = line.strip().split('\t')
        if line[0] == ids:
            x+=1
            vectors = line[1].split(' ')

            embeddingvectors.append(vectors[0:129]) #str
            print(len(vectors))
            
f2.close()
print(x)#390

390
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128


In [47]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/abstract2vec.txt","w")
 
for line in embeddingvectors:
    for item in line:
        f.write(item + ' ')
    f.write('\n')
f.close()

# journal IF matrix

In [48]:
df = pd.read_excel(io='/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/if2017.xlsx')
Full_Title = df['Full Title'].str.lower()
IF = df['IF 2016-2017']
Full_Title = list(Full_Title)
IF = list(IF) #102

alljournalnames = []
print(len(alljournal)) #102
for jj in alljournal:
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/selectedJournals.txt','r')
    for line in tqdm(f) :
        line = line.strip().split('\t')
        if line[0] == jj:
            alljournalnames.append(line[1])
print(len(alljournalnames))

IF_journal = []
for name in alljournalnames:
    for title,ifs in zip(Full_Title,IF):
        if title == name:
            i = Full_Title.index(title)
            score = IF[i]
            IF_journal.append(score)
print(len(IF_journal)) #102

102


7623it [00:00, 1050712.43it/s]
7623it [00:00, 899158.56it/s]
7623it [00:00, 925123.10it/s]
7623it [00:00, 977832.88it/s]
7623it [00:00, 992185.55it/s]
7623it [00:00, 934423.81it/s]
7623it [00:00, 826693.02it/s]
7623it [00:00, 1066448.06it/s]
7623it [00:00, 1052926.94it/s]
7623it [00:00, 1001038.80it/s]
7623it [00:00, 985640.11it/s]
7623it [00:00, 961512.63it/s]
7623it [00:00, 911280.27it/s]
7623it [00:00, 896310.25it/s]
7623it [00:00, 968737.45it/s]
7623it [00:00, 1136379.71it/s]
7623it [00:00, 882505.64it/s]
7623it [00:00, 1051610.95it/s]
7623it [00:00, 932815.36it/s]
7623it [00:00, 930887.10it/s]
7623it [00:00, 1118412.60it/s]
7623it [00:00, 768050.62it/s]
7623it [00:00, 953683.09it/s]
7623it [00:00, 1058749.61it/s]
7623it [00:00, 769844.44it/s]
7623it [00:00, 1015989.18it/s]
7623it [00:00, 1060505.47it/s]
7623it [00:00, 1029301.08it/s]
7623it [00:00, 883212.60it/s]
7623it [00:00, 1085124.02it/s]
7623it [00:00, 832591.52it/s]
7623it [00:00, 1057139.34it/s]
7623it [00:00, 929290.80it/

102
102


In [49]:

f=open("/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/IFS.txt","w")
 
for line in IF_journal:
    f.write(str(line))
    f.write('\n')
f.close()

# author credit matrix

In [50]:
#author credit
df = pd.read_csv('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/inputTable.csv')
df_annomous = df.loc[df['label_ifAbnormal'] == 1]
dfa = []
dfb = []
dfa = df_annomous['paperA'].unique()
dfb = df_annomous['paperB'].unique()
creditpapers = []
creditpapers = list(dfa) + list(dfb) #int
creditpapers = sorted(set(creditpapers),key = creditpapers.index)
creditpaperstr = [str(x) for x in creditpapers]

In [51]:
annmousauthors = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/PaperAuthorAffiliations.txt','r')

for line in tqdm(f):
    line = line.split('\t')
    if line[0] in creditpaperstr:
        annmousauthors.append(line[1]) #str

f.close()
print(len(annmousauthors)) #2014

455999370it [36:26, 208544.00it/s]

2014


In [69]:

f=open("/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/annmousauthors.txt","w")
for line in annmousauthors:
    f.write(line)
    f.write('\n')
f.close()

In [52]:
annmousauthorslist = sorted(set(annmousauthors))
print(len(annmousauthorslist)) #1779

1779


In [53]:
matrixcredic = [[1 for i in range(2)] for j in range(len(allauthor))]
#matrixcredic.shape
i = 0
h = 0
for i in range(len(allauthor)):
    matrixcredic[i][0] = allauthor[i]
    i += 1
for item in matrixcredic:
    if str(item[0]) in annmousauthors:
        item[1] = 0
        h += 1
credit = []
for item in matrixcredic:
    credit.append(item[1])#int
print(len(credit)) #2566
print(h) #76

2566
76


In [54]:
creditstr = [str(x) for x in credit]
f=open("/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/authorcredit.txt","w")
 
for line in creditstr:
    f.write(line)
    f.write('\n')
f.close()

# journal paper matrix

In [57]:
journalpapermatrix = [[0 for i in range(len(allpaperidstr))] for j in range(len(alljournal))]

for ids,js in zip(allpaperidstr,alljournal):
    j = allpaperidstr.index(ids)
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed/paperjournal.txt', 'r')
    for line in f:
        line = line.strip().split(' ')
        if line[0] == ids:
            i = alljournal.index(js)
            journalpapermatrix[i][j] = 1

In [58]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/journalpaper.txt","w")
 
for line in journalpapermatrix:
    for item in line:
        f.write(str(item)+ ' ')
    f.write('\n')
f.close()

# author author matrix

In [59]:
coauthormatrix = [[0 for i in range(len(allauthor))] for j in range(len(allauthor))]
for paperid in allpaperidstr:
    authors = []
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed/paperauthor.txt', 'r')
    for line in f:
        line = line.strip().split(' ')
        if line[0] == paperid:
            authors.append(line[1])
    authors = sorted(set(authors))
    for author1 in authors:
        i = allauthor.index(author1)
        for author2 in authors:
            j = allauthor.index(author2)
            coauthormatrix[i][j] += 1

In [60]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/authorauthor.txt","w")
 
for line in coauthormatrix:
    for item in line:
        f.write(str(item)+ ' ')
    f.write('\n')
f.close()

# author paper matrix

In [61]:
print(len(citingpaperid)) #504
print(len(citedpaperid))#504
print(len(citinginfo1))#330

545
545
143


In [62]:
authorpapermatrix = [[0 for i in range(len(allpaperidstr))] for j in range(len(allauthor))]
allpaperid = [int(x) for x in allpaperid]
for a,b in zip(citingpaperid,citedpaperid):
    j = allpaperid.index(b)
    author = []
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed/paperauthor.txt', 'r')
    for line in f:
        line = line.strip().split(' ')
        if int(line[0]) == a:
            author.append(line[1])
    author = sorted(set(author))
    for au in author:
        i = allauthor.index(au)
        authorpapermatrix[i][j] = 1
    
f.close()

In [63]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/authorpaper.txt","w")
 
for line in tqdm(authorpapermatrix):
    for item in line:
        f.write(str(item)+' ')
    f.write('\n')
f.close()

100%|█████████████████████████████████████| 2566/2566 [00:00<00:00, 7613.50it/s]


# 计算引用链接数量和合作链接数量

In [64]:
sum(map(sum,authorpapermatrix)) #引用链接数量

4194

In [65]:
sum(map(sum,coauthormatrix))#合作链接数量

29633

In [66]:
s = 0
for i in range(len(allauthor)):
    s += coauthormatrix[i][i]
    i += 1
print(s)

h = (sum(map(sum,coauthormatrix)) - s)/2
print(h)

2759
13437.0
